Math 5750/6880: Mathematics of Data Science \
Project 3

# 1. Fashion-MNIST image classification using sklearn

In [3]:
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(len(X_train), -1)
X_test  = X_test.reshape(len(X_test), -1)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# Base Model
mlp_baseline = MLPClassifier(hidden_layer_sizes=(100,),
                             activation='relu',
                             solver='adam',
                             learning_rate_init=0.001,
                             max_iter=20,
                             random_state=42,
                             verbose=True)
start_time = time.time()
mlp_baseline.fit(X_train, y_train)
train_time = time.time() - start_time

# Evaluate
y_predict = mlp_baseline.predict(X_test)
acc = accuracy_score(y_test, y_predict)
cm = confusion_matrix(y_test, y_predict)

print(f"Baseline Accuracy: {acc:.4f}")
print(f"Training Time: {train_time:.2f} seconds")
print("Confusion Matrix:\n", cm)

Iteration 1, loss = 0.49535171
Iteration 2, loss = 0.34883078
Iteration 3, loss = 0.30860900
Iteration 4, loss = 0.27836648
Iteration 5, loss = 0.26046428
Iteration 6, loss = 0.24447223
Iteration 7, loss = 0.23107564
Iteration 8, loss = 0.21711418
Iteration 9, loss = 0.20663219
Iteration 10, loss = 0.19509654
Iteration 11, loss = 0.18666009
Iteration 12, loss = 0.17665766
Iteration 13, loss = 0.17017831
Iteration 14, loss = 0.16164371
Iteration 15, loss = 0.15204313
Iteration 16, loss = 0.14558696
Iteration 17, loss = 0.14179169
Iteration 18, loss = 0.13422242
Iteration 19, loss = 0.12726104
Iteration 20, loss = 0.12637073


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


Baseline Accuracy: 0.8804
Training Time: 46.51 seconds
Confusion Matrix:
 [[768   3  17  28   7   0 168   0   9   0]
 [  2 973   1  16   4   0   3   0   1   0]
 [ 13   1 781  14 116   1  74   0   0   0]
 [ 17  10  12 884  38   0  36   0   3   0]
 [  1   1  61  23 862   1  49   0   2   0]
 [  0   0   0   1   0 955   0  27   1  16]
 [ 79   2  80  27  74   0 729   0   9   0]
 [  0   0   0   0   0  16   0 972   0  12]
 [  9   0   6   3   7   5   8   4 957   1]
 [  0   0   1   0   0  10   1  63   2 923]]


# 3. Fashion-MNIST image classification  using pytorch

In [ ]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
import torch
from torch.utils.data import TensorDataset, DataLoader

# Load Fashion-MNIST
# Classes (0-9): T-shirt/top, Trouser, Pullover, Dress, Coat, Sandal, Shirt, Sneaker, Bag, Ankle boot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# scale to [0,1], add channel dimension -> (N, 1, 28, 28)
X_train = (X_train.astype("float32") / 255.0)[:, None, :, :]
X_test  = (X_test.astype("float32")  / 255.0)[:,  None, :, :]

y_train = y_train.astype(np.int64)
y_test  = y_test.astype(np.int64)

# train/val split: last 10k of train as validation
X_tr, X_val = X_train[:50000], X_train[50000:]
y_tr, y_val = y_train[:50000], y_train[50000:]

# wrap in PyTorch TensorDatasets and DataLoaders
train_ds = TensorDataset(torch.from_numpy(X_tr),  torch.from_numpy(y_tr))
val_ds   = TensorDataset(torch.from_numpy(X_val), torch.from_numpy(y_val))
test_ds  = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# In colab, you should ``change runtime type'' to GPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# your code here